In [1]:
import os
%pwd

'd:\\personal_files\\python\\end_to_End_nlp_projects\\1_text_summarization\\__github__\\text-summarization\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\personal_files\\python\\end_to_End_nlp_projects\\1_text_summarization\\__github__\\text-summarization'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [5]:
from textsummarization import constants
from textsummarization.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_file_path = constants.CONFIG_FILE_PATH, params_file_path = constants.PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_roots])
    
    def get_model_trainer_config(self):
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir, data_path = config.data_path, model_ckpt=config.model_ckpt, 
            num_train_epochs=params.num_train_epochs, warmup_steps=params.warmup_steps, 
            per_device_train_batch_size=params.per_device_train_batch_size, weight_decay=params.weight_decay, 
            logging_steps=params.logging_steps, evaluation_strategy=params.evaluation_strategy, 
            eval_steps=params.evaluation_strategy, save_steps=params.save_steps, 
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        return model_trainer_config

In [7]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

d:\personal_files\python\end_to_End_nlp_projects\1_text_summarization\__github__\text-summarization\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self, ):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print("model_ckpt:", self.config)
        print("model____:", self.config.model_ckpt)
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps, evaluation_strategy=self.config.evaluation_strategy,
        #     eval_steps=self.config.eval_steps, save_steps=self.config.eval_steps, gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500, per_device_train_batch_size=1,
            per_device_eval_batch_size=1, weight_decay=0.01, logging_steps=10, evaluation_strategy='steps', eval_steps=500,
            save_steps=1e6, gradient_accumulation_steps=16
        )

        trainer = Trainer(
            model = model_pegasus, args = trainer_args, tokenizer = tokenizer, data_collator=seq2seq_data_collator,
            train_dataset = dataset_samsum_pt["test"], eval_dataset = dataset_samsum_pt["validation"]

        )
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config = model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-07-08 13:01:55,906: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-08 13:01:55,906: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-08 13:01:55,906: INFO: common: Created directory at: artifacts]
[2023-07-08 13:01:55,906: INFO: common: Created directory at: artifacts/model_trainer]
model_ckpt: ModelTrainerConfig(root_dir='artifacts/model_trainer', data_path='artifacts/data_transformation/samsum_dataset', model_ckpt='google/pegasus-cnn_dailymail', num_train_epochs=1, warmup_steps=500, per_device_train_batch_size=1, weight_decay=0.01, logging_steps=10, evaluation_strategy='steps', eval_steps='steps', save_steps='1e6', gradient_accumulation_steps=16)
model____: google/pegasus-cnn_dailymail


d:\personal_files\python\end_to_End_nlp_projects\1_text_summarization\__github__\text-summarization\myenv\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 20%|█▉        | 10/51 [07:54<33:15, 48.66s/it]

{'loss': 3.3137, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [15:21<23:08, 44.78s/it]

{'loss': 3.1, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 2.9817, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 98%|█████████▊| 50/51 [37:08<00:39, 39.70s/it]

{'loss': 3.1026, 'learning_rate': 5e-06, 'epoch': 0.98}


100%|██████████| 51/51 [38:02<00:00, 44.75s/it]


{'train_runtime': 2282.2377, 'train_samples_per_second': 0.359, 'train_steps_per_second': 0.022, 'train_loss': 3.1163070435617484, 'epoch': 1.0}
